# SPHINCS+

[pdf file](../related_work/sphincs+-r3.1-specification.pdf)


## 1. Introduction

- Lamport's one-time signatures [15] → extended by Merkle [17] for many-time use.
- Relies on simple hash function properties; Merkle’s method originally required a fixed number of signatures and was stateful.
- Over time, performance and theoretical underpinnings improved, leading to XMSS [7,8] (still stateful).
- SPHINCS [5] introduced a stateless hash-based signature (larger keys/signatures vs. XMSS).
- SPHINCS+ further refines SPHINCS. It uses **hypertrees** and few-time signature (FTS) schemes to authenticate many key pairs.


For each new message, a (pseudo)random **FTS** key pair is chosen to sign the message. More specifically, a hypertree is a tree of hash-based many-time signatures (**MTS**).


An MTS signature is just a classical Merkle-tree signature in the case of SPHINCS+. It consists of a one-time signature (OTS) on the given message plus the authentication path in the binary hash-tree, authenticating the $N$ OTS key pairs of one MTS key pair.

The **public key** of SPHINCS+ is essentially the public key of the top level MTS which is just the **root node** of its binary hash tree and hence, a single hash value.


The SPHINCS+ secret key is just a single secret seed value. From this, all the OTS and FTS secret keys are generated in a pseudorandom manner.


## 1.2 Organization

We start with basic notation. Afterwards we define WOTS+, the **OTS** used in SPHINCS+. Next, we specify XMSS, the **MTS** used in SPHINCS+, and how it is used to do HT signatures. Then, we define FORS, the **FTS** used, to finally specify SPHINCS+.


### 2.7.1. Tweakable Hash Functions

A tweakable hash function takes a public seed **PK.seed** and context information in form of an address **ADRS** in addition to the message input.


The schemes described in this specification build upon several instantiations of tweakable hash functions of the form:

$T_\ell: B^n \times B^{32} \times B^{\ell n} \rightarrow B^n$

$md \leftarrow T_\ell(\text{PK.seed}, \text{ADRS}, M)$

mapping an $\ell n$-byte message $M$ to an $n$-byte hash value $md$ using an $n$-byte seed $\text{PK.seed}$ and a 32-byte address $\text{ADRS}$. The function $T_\ell$ is denoted by `T_l` in pseudocode.


# 6.SPHINCS+

## 6.2. SPHINCS+ Key Generation

The SPHINCS+ private key contains two elements:

1. The $n$-byte secret seed `SK.seed` which is used to generate all the WOTS+ and FORS private key elements
2. An $n$-byte PRF key `SK.prf` which is used to deterministically generate a randomization value for the randomized message hash


Algorithm 19: SPHINCS+ Key Pair Generation

Function $\texttt{spx\_keygen}$ generates a SPHINCS+ key pair $(SK,PK)$:

```python
# filepath: algorithms/spx_keygen.py
def spx_keygen():
    SK.seed = sec_rand(n)
    SK.prf = sec_rand(n)
    PK.seed = sec_rand(n)
    PK.root = ht_PKgen(SK.seed, PK.seed)
    return ((SK.seed, SK.prf, PK.seed, PK.root),
            (PK.seed, PK.root))
```


## 6.3 SPHINCS+ Signature

A SPHINCS+ signature $\text{SIG}_{HT}$ is a byte string of length $(1+k(a+1)+h+d\text{len})n$. It consists of:

- An $n$-byte randomization string $R$
- A FORS signature $\text{SIG}_{FORS}$ consisting of $k(a+1)$ $n$-byte strings
- A HT signature $\text{SIG}_{HT}$ of $(h+ d\text{len})n$ bytes


## 6.4. SPHINCS+ Signature Generation

concise algorithm for SPHINCS+ signature generation:

- init the parameters
- compute the message digest `md` by the `H_msg`
- use the `md` and ADRS, by the tree from the `md`,to generate the FORS signature `fors_sign`
- creat the FORS public key
- use the HT to generate the HT signature `ht_sign`, also by the tree from the `md`


NOTE : this part context from other paper.

LATENCY (µs) OF HASH FUNCTIONS FOR SPHINCS+-128F-SIMPLE

| Function                | Latency |
| ----------------------- | ------- |
| $T_\ell(\text{wots})$   | 19.9    |
| $T_\ell(\text{fors})$   | 18.7    |
| $H$                     | 3.2     |
| $F$                     | 2.8     |
| $\text{PRF}$            | 1.6     |
| $\text{PRF}_\text{msg}$ | 5.9     |
| $H_\text{msg}$          | 4.8     |
